# Iron Supported, Nitrogen-doped, Graphene (And Related Systems) <a name="head"></a>
# Descriptor Scaling
***

## Import Modules <a name="import-modules"></a>

In [1]:
# Setting Custom Paths ********************************************************
# *****************************************************************************
import os
import sys

sys.path.insert(0, os.path.join(
        os.environ["PROJ_fe_graph"],
        "data"))

sys.path.insert(0, os.path.join(
        os.environ["PROJ_fe_graph"],
        "workflow"))

# Python Modules **************************************************************
# *****************************************************************************
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

from ase.visualize import view

import plotly.graph_objs as go
import plotly.plotly as py

from sklearn.linear_model import LinearRegression

# My Modules ******************************************************************
# *****************************************************************************
from colors.colors import color_list


# Local Imports ***************************************************************
# *****************************************************************************
from proj_data_fe_graph import (
    system_color_map_tmp,
    system_color_map,
    proj_dir_name,
    )

from an_data_processing import load_df, load_vegge_data

from methods import point_plot_settings

Divisor is Energy class instance!!!


## Script Inputs

In [2]:
save_plot = True

marker_size = 12

line_x_array = [-0.5, 3.5]
# color_list = ["red", "blue"]

border_color_list = [
    "red",
    "blue",
    "green",
    "magenta",
    ]

## Construct Dataframe

In [3]:
df = load_df(from_file=True, root_dir="..")

df = df[df["adsorbate"] != "h2o"]
# df = df[df["adsorbate"] != "oh"]
# df = df[df["system"] != "Fe_slab"]
# df = df[df["system"] != "graphene"]

df = df[[
    "ads_e",
    "adsorbate",
    "site",
    "spinpol",
    "system",
    "notes"
    ]]

df.set_index(["site", "system", "spinpol"], inplace=True)

df = df.pivot(index=None, columns="adsorbate")

df_1 = df.xs("ads_e", axis=1, drop_level=True)
df_2 = df.xs("notes", axis=1, drop_level=True)

# df_2 = df_2.rename(index=str, columns={
df_2 = df_2.rename(columns={
    "o": "o-notes",
    "oh": "oh-notes",
    "ooh": "ooh-notes",
    })

df["index"] = df.index.values
df["index"] = df["index"].astype(str)

df = pd.concat([df_1, df_2], axis=1, join_axes=[df_1.index])

df_master = df.copy()
# text_i = df["oh-notes"].tolist()

df_master["notes"] = df_master["o-notes"] + " | " + df_master["oh-notes"] + " | " + df_master["ooh-notes"]

Attempting to load df from pickle


# Linear Regression to Adsorbed Species

# Plotting Scaling Data Series

In [4]:
trace_list = []

## OOH vs OH

In [5]:
tmp_bool = False
for i_ind, (index, row) in enumerate(df_master.iterrows()):
    plot_settings_dict = point_plot_settings(
        i_ind,
        index,
        row,
        border_color_list=border_color_list,
        system_color_map=system_color_map_tmp,
        )

    show_leg = False
    if tmp_bool is False:
        if plot_settings_dict["marker_symbol"] == "circle":
            tmp_bool = True
            show_leg = True

            trace_i = go.Scatter(
                y = [row["ooh"]],
                x = [row["oh"]],
                hoverinfo="text",
                text=plot_settings_dict["hover_text"],
                mode='markers',
                name = "*OOH scaling",
                legendgroup="*OOH scaling",
                showlegend=show_leg,
                marker=dict(
                    size=marker_size,
                    symbol=plot_settings_dict["marker_symbol"],
#                     color = color_list[0],
                    color=plot_settings_dict["sys_color"],
                )
            )
            trace_list.append(trace_i)

    trace_i = go.Scatter(
        y = [row["ooh"]],
        x = [row["oh"]], 
        hoverinfo="text",
        text=plot_settings_dict["hover_text"],
        mode='markers',
        name = "*OOH scaling",
        legendgroup="*OOH scaling",
#         showlegend=show_leg,
        showlegend=False,
        marker=dict(
            size=marker_size,
            symbol=plot_settings_dict["marker_symbol"],
#             color=color_list[0],
            color=plot_settings_dict["sys_color"],

            line=dict(
#                 color=plot_settings_dict["marker_line"],
#                 color=plot_settings_dict["sys_color"],
                color="black",
                width=1.5,
            )
        )
    )
    trace_list.append(trace_i)

## O vs OH

In [6]:
tmp_bool = False
for i_ind, (index, row) in enumerate(df_master.iterrows()):
    plot_settings_dict = point_plot_settings(
        i_ind,
        index,
        row,
        border_color_list=border_color_list,
        system_color_map=system_color_map_tmp,
        )
#     plot_settings_dict = point_plot_settings(i_ind, index, row)
 
    show_leg = False
    if tmp_bool is False:
        if plot_settings_dict["marker_symbol"] == "circle":
            tmp_bool = True
            show_leg = True

            trace_i = go.Scatter(
                y = [row["o"]],
                x = [row["oh"]],
                hoverinfo="text",
                text=plot_settings_dict["hover_text"],
                mode='markers',
                name = "*O scaling",
                legendgroup="*O scaling",
                showlegend=True,
                marker=dict(
                    size=marker_size,
                    symbol=plot_settings_dict["marker_symbol"],
#                     color=color_list[1],
                    color=plot_settings_dict["sys_color"],
                )
            )            
            trace_list.append(trace_i)

    trace_i = go.Scatter(
        y = row[["o"]],
        x = row[["oh"]],
        hoverinfo="text",
        text=plot_settings_dict["hover_text"],
        mode='markers',
        name = "*O scaling",
        legendgroup="*O scaling",
        showlegend=False,
        marker=dict(
            size=marker_size,
            symbol=plot_settings_dict["marker_symbol"],
#             color=color_list[1],
            color=plot_settings_dict["sys_color"],
            line=dict(
#                 color=plot_settings_dict["marker_line"],
                color="red",
#                 width=2,
                width=0.5,
            )
        )
    )
    trace_list.append(trace_i)

## Ideal OOH and O vs OH Scaling

In [7]:
# Ideal OOH vs OH Scaling Line
x = line_x_array
trace_ooh_scaling = go.Scatter(
    x = x,
    y = [x[0] + 3.2, x[1] + 3.2],
    name = "ooh v oh (ideal)",
    mode='lines',
    opacity=0.6,
    line = {
        "dash": "dot",
        "color": "black",
    },
)
# trace_list.append(trace_ooh_scaling)
trace_list.insert(0, trace_ooh_scaling)

# Ideal O vs OH Scaling Line
x = line_x_array
trace_o_scaling = go.Scatter(
    x = x,
    y = [2 * x[0], 2 * x[1]],
    name = "o v oh (ideal)",
    mode='lines',
    opacity=0.6,
    line = {
        "dash": "dot",
        "color": "red",
    },
)

# trace_list.append(trace_o_scaling)
trace_list.insert(0, trace_o_scaling)

## Fit Lines

Before fitting the scaling relations, bad data should be diregarded.
This includes the following situations:
* Desorption: The adsorbate molecule does not bind with the surface
* Dissociation: The adsorbate is "broken up" by the surface

In [8]:
df_regr = df_master.copy()

df_regr = df_regr[df_regr["o-notes"] != "Desorbed"]
df_regr = df_regr[df_regr["oh-notes"] != "Desorbed"]
df_regr = df_regr[df_regr["ooh-notes"] != "Desorbed"]

df_regr = df_regr[df_regr["o-notes"] != "Dissociated"]
df_regr = df_regr[df_regr["oh-notes"] != "Dissociated"]
df_regr = df_regr[df_regr["ooh-notes"] != "Dissociated"]

In [9]:
df_regr_ooh_v_oh = df_regr.copy()
df = df_regr_ooh_v_oh
# Removing row with NA values for OOH or OH
df = df[np.isfinite(df["ooh"])]
df = df[np.isfinite(df["oh"])]

lr = LinearRegression()

y_data = np.array(df["ooh"].tolist())
x_data = np.array(df["oh"].tolist()).reshape(-1, 1)

lr.fit(x_data, y_data)

b = lr.intercept_
m = lr.coef_[0]
print(b)
print(m)
b_ooh = b
m_ooh = m

print("OOH Fit")
print("intercept (b) and slope (m) coefficients:")


x_array = np.array(line_x_array)
y_pred = lr.predict(x_array.reshape(-1, 1))

trace_ooh_fit = go.Scatter(
    x = x_array,
    y = y_pred,
    name = "*OOH vs *OH (fitted)",
    mode='lines',
    line = {
        "color": "black",
    },
)

# trace_list.append(trace_ooh_fit)
trace_list.insert(0, trace_ooh_fit)
# **************************************************

df_regr_o_v_oh = df_regr.copy()
df = df_regr_o_v_oh
# Removing row with NA values for OOH or OH
df = df[np.isfinite(df["o"])]
df = df[np.isfinite(df["oh"])]

lr = LinearRegression()

y_data = np.array(df["o"].tolist())
x_data = np.array(df["oh"].tolist()).reshape(-1, 1)

lr.fit(x_data, y_data)

b = lr.intercept_
m = lr.coef_[0]
b_o = b
m_o = m

print("O Fit")
print("intercept (b) and slope (m) coefficients:")
print(b)
print(m)


x_array = np.array(line_x_array)
y_pred = lr.predict(x_array.reshape(-1, 1))

trace_o_fit = go.Scatter(
    x = x_array,
    y = y_pred,
    name = "*O vs *OH (fitted))",
    mode='lines',
    line = {
        "color": "red",
    },
)

# trace_list.append(trace_o_fit)
trace_list.insert(0, trace_o_fit)

2.9177148903311965
1.2988345180866383
OOH Fit
intercept (b) and slope (m) coefficients:
O Fit
intercept (b) and slope (m) coefficients:
0.5017142485515365
1.8144171589792406


# Plotting

In [10]:
tick_lab_size = 9 * (4. / 3.)
axes_lab_size = 11 * (4. / 3.)

common_axis_dict = {
    "zeroline": False,
    "showline": True,
    "mirror": 'ticks',
    "linecolor": 'black',
    "showgrid": False,
    "titlefont": dict(
        size=axes_lab_size,
        color="black",
        ),
    "tickfont": dict(
        size=tick_lab_size,
        color="black",
        ),
    "ticks": 'inside',
    "tick0": 0,
    "tickcolor": 'black',
    # "dtick": 0.25,
    "ticklen": 2,
    "tickwidth": 1,
    }

layout = dict(
#     title = 'OOH, OH, O Scaling Relations',
   
    xaxis=dict(
        common_axis_dict,
        **{
            "title": "dG<sub>*OH</sub> (eV)",
            "dtick": 0.5,
            "range": line_x_array,
            }
    ),

    yaxis=dict(
        common_axis_dict,
        **{
            "title": "dG<sub>*OOH</sub>, dG<sub>*O</sub> (eV)",
            }
        ),

    width=1. * 18.7 * 37.795275591,
    height=0.75 * 18.7 * 37.795275591,

#     width=200 * 4.,
#     height=200 * 3.,

    )

In [11]:
border_coord = {
    "x0": 0.01,
    "y0": 0.8,
    "x1": 0.35,
    "y1": 1.,
    }

## Annotations

In [12]:
eqn_ooh = "" \
    + "G<sub>*OOH</sub> = " \
    + "{:.4f}".format(m_ooh) \
    + "*" \
    + "G<sub>*OH</sub>" \
    + " + " \
    + "{:.4f}".format(b_ooh) \
    + ""

eqn_o = "" \
    + "G<sub>*O</sub> = " \
    + "{:.4f}".format(m_o) \
    + "*" \
    + "G<sub>*OH</sub>" \
    + " + " \
    + "{:.4f}".format(b_o) \
    + ""

eqn_pos = {
    "x": 0.01,
    "y": 0.99,
    }

annotations=[
    dict(
        font=dict(family='Arial', size=12, color='white'),
#         x=0.1,
#         y=0.8,

        x=eqn_pos["x"],
        y=eqn_pos["y"],

        xref='paper',
        yref='paper',
        text=eqn_ooh,
        showarrow=False,
        arrowhead=7,
        ax=0,
        ay=-2,
        bgcolor="black",
        bordercolor="black",
        borderpad=5,
        yshift=0.,
        ),

    dict(
        font=dict(family='Arial', size=12, color='white'),
        x=eqn_pos["x"],
        y=eqn_pos["y"],

#         x=0.1,
#         y=0.8,
        xref='paper',
        yref='paper',
        text=eqn_o,
        showarrow=False,
        arrowhead=7,
        ax=0,
        ay=-2,
        bgcolor="black",
        bordercolor="black",
        borderpad=5,
        yshift=-30.,
        ),
    ]

In [13]:
if save_plot:
    save_dir = proj_dir_name
else:
    save_dir = "__temp__"

In [14]:
layout["annotations"] = annotations

fig = dict(data=trace_list, layout=layout)
py.iplot(fig,
    filename=os.path.join(
        save_dir,
        "scaling_relations",
        "pl_scaling_relations_w_legend",
        ),
    )

In [15]:
layout["showlegend"] = False

fig = dict(data=trace_list, layout=layout)

py.iplot(fig,
    filename=os.path.join(
        save_dir,
        "scaling_relations",
        "pl_scaling_relations_no_legend",
        ),
    )